<a href="https://colab.research.google.com/github/acedesci/scanalytics/blob/master/S8_9_retail_analytics/S09_Bass_Diffusion_Full_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# S9 - Bass Diffusion - full demo

---

In session 2, we have defined the following two functions for Bass diffusion model:

<blockquote>
The cumulative number of adoptors (people who bought/adopted the product) $S(t)$ from the beginning (time 0) to time $t$ is described by $S(t) = m\times F(t)$ where $m$ is the market size (the estimated total number of potential customers) and $F(t)$ is the <i>cumulative</i> probability that an individual (potential customer) has already adopted the product by time $t$ (which needs to be calculated). 
We can obtain $F(t)$ as a function of $p$ and $q$ as follows:

$$F(t) = \frac{1-e^{-(p+q)t}}{1+\frac{q}{p}e^{-(p+q)t}} $$

where 
<br /> 
* $p$ is the coefficiant of innovation
<br /> 
* $q$ is the coefficient of imitation
<br /> 

In addition, the probability of adoption by an individual at time $t$ which is represented by $f(t)$ (recall that $f(t) = \frac{d}{dt}F(t)$) can be calculated as follows: 

$$f(t) = \frac{d}{dt}F(t) = \frac{e^{((p+q)t)}p(p+q)^2}{[pe^{((p+q)t)}+q]^2} $$.

Consequently, the estimated number of adoptions in each period can be calculated as  $s(t) = m\times f(t)$

</blockquote>

We have then prepared the following two functions to calculate $F(t)$ and $f(t)$.

In [ ]:
import math
def Bass_cumulative_probability_Ft(p, q, t):
    return (1-math.exp(-(p+q)*t))/(1+(q/p)*math.exp(-(p+q)*t))

def Bass_probability_ft(p, q, t):
    return (math.exp((p+q)*t)*p*(p+q)**2)/(p*math.exp((p+q)*t)+q)**2

Based on the values of $p$, $q$ and $m$ of the quarterly sales of iPhone provided in the code cell below, please compute the values of the estimated probability of adoption $f(t)$, cumulative probability of adoption $F(t)$, the number of adoptions $s(t) = m\times f(t)$, and cumulative number of adoptions $S(t) = m\times F(t)$ for each quarter, which is indexed by $t$. Note that the first column below is an index whereas the subsequent four columns contain the corresponding values of the estimations based on Bass diffusion model.


|Quarter_Index| ft | Ft| st | St | 
| :- | :- | :- | :- | :- | 
|...| ... |...| ... |...|




### Regression for Bass parameters estimation (this is the additional part to the original notebook in S4)
**Estimating Bass diffusion model parameters**

We can now complete the Bass diffusion model pipeline by first estimating the parameters $p$, $q$ and $m$ from the data prior to performing prediction. First we load the modules which are required as well as the actual data of sales of iPhone. The regression model for parameters estimation is presented in the paper below (Section "Discrete Analogue").

<b>Reference</b>: Bass, F. M. (1969). A new product growth for model consumer durables. Management science, 15(5), 215-227.


In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import *

url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/S04_Data_Structures_2/iphone_quarter_sales.csv'
actual_sales = pd.read_csv(url, index_col='Quarter')
actual_sales.head()

We would first determine the coefficients for the Bass diffusion model in order to perform the prediction as seen in the previous session. The Bass model in a basic form, can be written as:

$s(t) = pm + (q-p)S(t) - \frac{q}{m}S(t)^2$

which can be solved by a regression function. Here we will use the following equation (note that we do not shift one period like in the paper since the period is a bit long).

$s(t) = a + bS(t) + cS(t)^2$

Correspondingly, we can obtain $m = \frac{-b \pm \sqrt{b^2-4ac}}{2c}$ ($m$ must be positive), $p = a/m$ and $q = -mc$ (for the detailed proof, you can review the Section "Discrete Analogue" in the paper).

In the steps below, we demonstrate how this can be done using the linear regression model.

In [ ]:
input = pd.DataFrame()
# we need to prepare the Y inputs from sales by taking the cumulative sum up until period T - 1

input["sales"] = actual_sales["Sales"]
input["cumulative_sales"] = np.cumsum(actual_sales["Sales"])
input["cumulative_sales_sq"] = [x**2 for x in input["cumulative_sales"]]
input.head()

Now we can use linear regression to determine $a$, $b$ and $c$ of the regression function above.

In [ ]:
X = input[["cumulative_sales","cumulative_sales_sq"]]
y = input["sales"]
reg = sklearn.linear_model.LinearRegression().fit(X,y)
print("intercept: "+str(reg.intercept_))
print("coefficients: "+str(reg.coef_))

a = reg.intercept_
b = reg.coef_[0]
c = reg.coef_[1]

Following the results, we can obtain

$m = \frac{-b \pm \sqrt{b^2-4ac}}{2c}$

and $m$ (the size of potential customer base) must be positive.

In [ ]:
# determine m
m1 = (-b+math.sqrt(b**2-4*a*c))/(2*c)
m2 = (-b-math.sqrt(b**2-4*a*c))/(2*c)
m = max(m1,m2)
print("m1 = ", m1, ", m2 = ", m2, "m = ", m)

Then, we can directly obtain $p = a/m$ and $q = -mc$ for the Bass diffusion model.

In [ ]:
p = a/m
q = -m*c
print("m = ", m, ", p = ", p, "q = ", q)

### The following codes are from Session 4 (Bass diffusion prediction from the given parameters)

**Question 1**: Please compute the values as indicated in the table above for the first 40 quarters of the sales (i.e., 10 years) using the parameters provided. The index of the quarter must be from $0 \rightarrow 39$ (i.e., the first quarter is considered period $t=0$).

In [ ]:
import pandas as pd
column_names = ['ft','Ft','st','St']
quarter_index = list(range(40))
bass_df = pd.DataFrame(index = quarter_index, columns = column_names)
bass_df.index.name = 'Quarter'

### start your code here ###
bass_df['ft'] = [Bass_probability_ft(p, q, t) for t in bass_df.index]
bass_df['Ft'] = [Bass_cumulative_probability_Ft(p, q, t) for t in bass_df.index]
bass_df['st'] = [m*bass_df.at[t,'ft'] for t in bass_df.index]
bass_df['St'] = [m*bass_df.at[t,'Ft'] for t in bass_df.index]
### end your code here ###

bass_df


**Question 2**: Please (i) plot the columns `['ft','Ft']` in a single plot using line plot (parameter `kind='line'`) and (ii) plot the columns `['st','St']` in a single plot using bar plot (parameter `kind='bar'`). You can also indicate the plot size using the parameter `figsize = (12,4)`.

In [ ]:
### start your code here ###
bass_df[['ft','Ft']].plot(figsize = (12,4))
bass_df[['st','St']].plot(kind = 'bar', figsize = (12,4))
### end your code here ###

Now we can compare the results with the actual quarterly sales of iPhone for the first 40 quarters (starting from Q2 2007 which the product was first introduced). 

**Question 3**: Please add the following columns and their corresponding values to the `bass_df` DataFrame


*   Column `'Actual'`: which contains the actual sales from the DataFrame `actual_sales`
*   Column `'CumulativeActual'`: which calculates the sum from the first quarter up until each quater in the index
*   Column `'PError'`: which calculates the percentage error of each corresponding quarter from column `'Actual'` and column `'st'` (estimated sales in each quarter), i.e., `PError = (Actual-st)/st`



In [ ]:
### start your code here ###
bass_df['Actual'] = actual_sales['Sales']
bass_df['CumulativeActual'] = [sum(actual_sales.loc[:t,'Sales']) for t in actual_sales['Sales'].index]
bass_df['PError'] = (bass_df['Actual'] - bass_df['st'])/bass_df['Actual']
### end your code here ###
bass_df

We can also generate the date of the last day of each quarter using the code below. This list will be later used as new indexes.

In [ ]:
quarter_index_date = pd.date_range('4/1/2007', periods=40, freq='Q')
quarter_index_date


**Question 4**: Please (i) replace the original indexes using the newly created list `quarter_index_date` and (ii) then create the following plots from the DataFrame `bass_df` using the size `figsize=(12,4)`:


*   Columns `['St','CumulativeActual']` using a line plot
*   Columns `['st','Actual']` using a line plot
*   Column `['PError']` using a bar plot

**Hint**: you can use the method `.set_index(...)` by calling `bass_df =  bass_df.set_index(new_index_list, index_name)` to set the index (see [link](https://www.w3resource.com/pandas/dataframe/dataframe-set_index.php))



In [ ]:
### start your code here ###
bass_df = bass_df.set_index(quarter_index_date, 'Quarter')
bass_df[['St','CumulativeActual']].plot(figsize=(12,4))
bass_df[['st','Actual']].plot(figsize=(12,4))
bass_df[['PError']].plot(kind='bar',figsize=(12,4))
### end your code here ###